In [2]:
from inference import SparseModel

/home/generall/projects/vector_search/tmp/msmacro/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model = SparseModel()

In [4]:
def read_queries():
    with open("data/qrels.train.with_queries.tsv", 'r') as file:
        next(file)
        for line in file:
            _, doc_id, query = line.strip().split("\t")
            yield int(doc_id), query

In [5]:
QDRANT_URL='https://7be44f8c-f347-4cf0-9a06-e65c5882815a.germanywestcentral-0.azure.cloud.qdrant.io:6333'
QDRANT_API_KEY='A-AlRH2U8J0rU9lv6rtbItTd7el3kyVq_7ftY3dKhfB3bBkuz3cAdA'
COLLECTION_NAME="msmarco"

In [6]:
from qdrant_client import QdrantClient, models

In [7]:
def conver_sparse_vector(sparse_vector: dict) -> models.SparseVector:
    indices = []
    values = []

    for (idx, value) in sparse_vector.items():
        indices.append(int(idx))
        values.append(value)

    return models.SparseVector(
        indices=indices,
        values=values
    )

In [8]:
client = QdrantClient(QDRANT_URL, api_key=QDRANT_API_KEY)

In [11]:
n = 0
hits = 0
limit = 100
number_of_queries = 1000

for doc_id, query in read_queries():
    n += 1
    if n > number_of_queries:
        break
    print(f"Processing query: {query}")
    sparse_vector = next(model.encode([query]))
    sparse_vector = conver_sparse_vector(sparse_vector)
    result = client.search(
        collection_name=COLLECTION_NAME,
        query_vector=models.NamedSparseVector(
            name="attention",
            vector=sparse_vector
        ),
        with_payload=True,
        limit=limit
    )
    
    found_ids = []

    for hit in result:
        found_ids.append(hit.id)

    if doc_id in found_ids:
        hits += 1

print(f"Recall @ {limit}: {hits} out of {n} = {hits/n}")


Processing query: )what was the immediate impact of the success of the manhattan project?
Processing query: _________ justice is designed to repair the harm to victim, the community and the offender caused by the offender criminal act. question 19 options:
Processing query: what color is amber urine
Processing query: is autoimmune hepatitis a bile acid synthesis disorder
Processing query: elegxo meaning
Processing query: how much does an average person make for tutoring
Processing query: can you use a calculator on the compass test
Processing query: what does physical medicine do
Processing query: what does pending mean on listing
Processing query: feeding rice cereal how many times per day
Processing query: most dependable affordable cars
Processing query: lithophile definition
Processing query: what is a flail chest
Processing query: what causes ear infection in adults
Processing query: put yourself on child support in texas
Processing query: mushrooms health benefits
Processing quer

In [9]:
client.search(
    collection_name=COLLECTION_NAME,
    query_vector=models.NamedSparseVector(
        name="attention",
        vector=conver_sparse_vector(next(model.encode([query])))
    ),
    with_payload=True,
    limit=3
)

[ScoredPoint(id=4153976, version=194717, score=0.24604332, payload={'text': 'You want to check the Map from Times Square to Manhattan! Apart from the trip distance, do you need road driving directions? Refer the Directions from Times Square to Manhattan! How much would be the distance if one could just fly from Times Square to Manhattan like a bird in a staight line? Check the Flight distance between Times Square and Manhattan. Travel time is an important point to know with the driving distance. Hence you might also want to know the Travel Time from Times Square to Manhattan. This will help you estimate how much time you will spend travelling for the distance from Times Square to Manhattan. Want to better plan this Times Square Manhattan distance? Get to know how to Travel from Times Square to Manhattan. Do you find travelling by road tiring? Want to know how much time is required to fly from Times Square to Manhattan? Check the Flight Time from Times Square to Manhattan. Are you looki